# Converting to TF Lite

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [2]:
print(tf.__file__)
print(tf.__version__)

/home/erick/miniconda3/envs/ml-zoomcamp/lib/python3.11/site-packages/tensorflow/__init__.py
2.14.0


In [3]:
MODEL_PATH: str = 'xception_v4_08_0.894.h5'

In [4]:
model = keras.models.load_model(MODEL_PATH)

In [10]:
!wget https://contents.mediadecathlon.com/p2163297/k$ee97c140e7a499e050c27b2128392ac6/pantalon-de-mujer-para-trekking-en-el-desierto-anti-rayos-uv-ecodisenado-desert-900-beige.jpg -O pants.png

--2023-11-08 15:16:12--  https://contents.mediadecathlon.com/p2163297/k/pantalon-de-mujer-para-trekking-en-el-desierto-anti-rayos-uv-ecodisenado-desert-900-beige.jpg
Resolving contents.mediadecathlon.com (contents.mediadecathlon.com)... 181.54.160.81, 181.54.160.9
Connecting to contents.mediadecathlon.com (contents.mediadecathlon.com)|181.54.160.81|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 206457 (202K) [image/jpeg]
Saving to: ‘pants.png’

pants.png           100%[===================>] 201.62K  --.-KB/s    in 0.02s   

2023-11-08 15:16:12 (8.33 MB/s) - ‘pants.png’ saved [206457/206457]



In [19]:
img = load_img('pants.png', target_size=(299,299))
X = np.array(img)
X = np.expand_dims(X, 0)
X = preprocess_input(X)

In [22]:
classes = [
    'dress',
    'hat',
    'longsleeve',
	'outwear',
    'pants',
	'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt']

In [27]:
pred = model.predict(X)[0]
pred

1/1 [==============================] - 0s 230ms/step


array([-1.7161683, -2.4631023, -1.2729362,  1.1031336,  7.054929 ,
        0.9493518, -2.377652 ,  1.9362751, -3.6682243, -3.1740787],
      dtype=float32)

In [29]:
sorted(zip(classes, pred), key=lambda x:-x[1])

[('pants', 7.054929),
 ('shorts', 1.9362751),
 ('outwear', 1.1031336),
 ('shirt', 0.9493518),
 ('longsleeve', -1.2729362),
 ('dress', -1.7161683),
 ('shoes', -2.377652),
 ('hat', -2.4631023),
 ('t-shirt', -3.1740787),
 ('skirt', -3.6682243)]

## Using TF-Lite

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [33]:
tflite_model = converter.convert()

# Save the model.
with open('clothing_model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpcvxm4tto/assets


INFO:tensorflow:Assets written to: /tmp/tmpcvxm4tto/assets
2023-11-08 15:38:21.329563: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-08 15:38:21.330385: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-08 15:38:21.357309: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpcvxm4tto
2023-11-08 15:38:21.377177: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-08 15:38:21.377252: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpcvxm4tto
2023-11-08 15:38:21.457789: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-11-08 15:38:21.481587: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-08 15:38:25.212775: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [35]:
!ls -lh

total 163M
-rwxrwxrwx 1 erick erick  81M Nov  8 15:38 clothing_model.tflite
-rwxrwxrwx 1 erick erick 202K Nov  8 15:16 pants.png
-rwxrwxrwx 1 erick erick 7.5K Nov  8 15:41 tensorflow-lite.ipynb
-rwxrwxrwx 1 erick erick  83M Nov  8 14:14 xception_v4_08_0.894.h5


In [37]:
import tensorflow.lite as tflite

In [38]:
# Load the TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

In [49]:
input_details = interpreter.get_input_details()
input_details

[{'name': 'serving_default_input_20:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [51]:
output_details = interpreter.get_output_details()
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 229,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [65]:
#X1 = tf.random.normal((1,299,299,3))
interpreter.set_tensor(input_details[0].get('index'), X)

In [66]:
interpreter.invoke()

In [67]:
pred = interpreter.get_tensor(output_details[0].get('index'))
pred

array([[-1.7161655, -2.463105 , -1.2729357,  1.1031358,  7.054927 ,
         0.9493514, -2.377652 ,  1.9362712, -3.6682208, -3.1740801]],
      dtype=float32)

In [68]:
list(zip(classes, pred[0]))

[('dress', -1.7161655),
 ('hat', -2.463105),
 ('longsleeve', -1.2729357),
 ('outwear', 1.1031358),
 ('pants', 7.054927),
 ('shirt', 0.9493514),
 ('shoes', -2.377652),
 ('shorts', 1.9362712),
 ('skirt', -3.6682208),
 ('t-shirt', -3.1740801)]

## Removing TF dependence

In [77]:
from PIL import Image

In [103]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.0
    return x

In [108]:
with Image.open('./pants.png') as img:
    img = img.resize((299,299), Image.NEAREST)

X = np.array(img, dtype='float32')
X = np.expand_dims(X, 0)

X = preprocess_input(X)

interpreter.set_tensor(input_details[0].get('index'), X)
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0].get('index'))
list(zip(classes, pred[0]))

[('dress', -1.7161655),
 ('hat', -2.463105),
 ('longsleeve', -1.2729357),
 ('outwear', 1.1031358),
 ('pants', 7.054927),
 ('shirt', 0.9493514),
 ('shoes', -2.377652),
 ('shorts', 1.9362712),
 ('skirt', -3.6682208),
 ('t-shirt', -3.1740801)]

## Simpler way

In [119]:
from keras_image_helper import create_preprocessor
import tensorflow.lite as tflite

In [121]:
# Load the TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

preprocessor = create_preprocessor('xception', (299,299))

In [122]:
X = preprocessor.from_path('pants.png')

interpreter.set_tensor(input_details[0].get('index'), X)
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0].get('index'))

list(zip(classes, pred[0]))

[('dress', -1.7161655),
 ('hat', -2.463105),
 ('longsleeve', -1.2729357),
 ('outwear', 1.1031358),
 ('pants', 7.054927),
 ('shirt', 0.9493514),
 ('shoes', -2.377652),
 ('shorts', 1.9362712),
 ('skirt', -3.6682208),
 ('t-shirt', -3.1740801)]

## Using offical TFLite Runtime

In [123]:
!python3 -m pip install tflite-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.1 MB/s eta 0:00:0000:0100:01


In [1]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
# Load the TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path='clothing_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

preprocessor = create_preprocessor('xception', (299,299))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
classes = [
    'dress',
    'hat',
    'longsleeve',
	'outwear',
    'pants',
	'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt']

In [5]:
X = preprocessor.from_path('pants.png')

interpreter.set_tensor(input_details[0].get('index'), X)
interpreter.invoke()
pred = interpreter.get_tensor(output_details[0].get('index'))

list(zip(classes, pred[0]))

[('dress', -1.7161655),
 ('hat', -2.463105),
 ('longsleeve', -1.2729357),
 ('outwear', 1.1031358),
 ('pants', 7.054927),
 ('shirt', 0.9493514),
 ('shoes', -2.377652),
 ('shorts', 1.9362712),
 ('skirt', -3.6682208),
 ('t-shirt', -3.1740801)]